In [1]:
from docuverse.engines import SearchResult
from docuverse.engines import SearchEngine
from docuverse.engines.search_engine_config_params import DocUVerseConfig
from docuverse.utils import get_param
import pickle

from docuverse.utils.elastic.elastic_ingestion import normalize_text

In [2]:
config="experiments/zdocs/milvus_dense.granite-125m.test.yaml"
engine=SearchEngine(DocUVerseConfig(config))

Retrieval engine: milvus-dense
Running on the gpus:  ['NVIDIA GeForce RTX 4090', 'NVIDIA GeForce RTX 5090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090']
=== done initializing model


In [3]:
results=engine.read_output()

In [4]:
res=results[0]

In [5]:
len(res)

35

In [6]:
urls=[[get_param(d, 'metadata.url') for d in res] for res in results]
qurls = [d.question.get('metadata')['gold-urls'] for d in results]
qtext = [d.question.text for d in results]

In [18]:
data=engine.read_data()

Cache filename is /home/raduf/.local/share/elastic_ingestion/benchmark__zdocs__corpus.jsonl.bz2_512_100_True_all_granite-embedding-125m-english.pickle.xz
Skipped 0 passages.


Computing statistics: 100%|██████████| 1167439/1167439 [00:02<00:00, 442971.14it/s]


Statistics:
Number of documents:1026632   
Number of tiles:    1167439   
#tiles per document:1.14      
                    Character-based:    Token-based:        
  Minimum length:   9                   6                   
  Maximum length:   2240                512                 
  Average length:   1084.2              269.6               
Char histogram:

# NumSamples = 1167439; Min = 9.00; Max = 2240.00
# Mean = 1084.212026; Variance = 406224.673368; SD = 637.357571; Median 977.000000
# each ∎ represents a count of 2632
    9.0000 -   232.1000 [ 55241]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  232.1000 -   455.2000 [197448]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  455.2000 -   678.3000 [153778]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  678.3000 -   901.4000 [133010]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  901.4000 -  1124.5000 [128056]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
 1124.5000 -  1347.6000 [ 91542]: ∎∎∎∎∎

In [ ]:
import re
VERSION_PATTERN = re.compile(r'(\d+(\.\d+)?(\.\d+)?)\?')

def normalize_url(url):
    url = VERSION_PATTERN.sub('latest?', url)
    return re.sub(r'#.*', '', url)

In [84]:
url2id = {}
id2pos = {}
reverse_norm_map = {}
def add_key(ddict, key1, key2):
    if key1 in ddict:
        # ddict[key1][key2] = 1
        if key2 in ddict[key1]:
            ddict[key1][key2] += 1
        else:
            ddict[key1][key2] = 1
    else:
        ddict[key1] = {key2: 1}

for pos, d in enumerate(data):
    url = d['metadata']['url']
    add_key(url2id, url, d['id'])
    norm_url = normalize_url(url)
    d['metadata']['norm_url'] = norm_url
    add_key(url2id, norm_url, d['id'])
    if norm_url in reverse_norm_map:
        if url in reverse_norm_map[norm_url]:
            pass
            # ids = url2id[norm_url].keys()
            # print(ids)
            # print(f"{norm_url} (position {pos} already seen in documents {ids}, at positions "
            #       f"{[id2pos[p] for p in list(ids) if p in id2pos]}.")
        else:
            reverse_norm_map[norm_url][url] = 1
        reverse_norm_map[norm_url][url] += 1
    else:
        reverse_norm_map[norm_url] = {url: 1}

    id2pos[d['id']] = pos

In [ ]:
num_errors = 0
for url in reverse_norm_map:
    ids = [id2pos[p] for p in url2id[url].keys()]
    for i, id in enumerate(ids):
        if ids[0]+i != id:
            print(f"Error with url {url}, id list {ids}, urls: {list(reverse_norm_map[url].keys())}")
            num_errors += 1
            break
print(f"Number of errors: {num_errors}")

In [30]:
from rouge_score.rouge_scorer import RougeScorer
rouge_scorer = RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
def compute_rouge_matches(result, thr=0.9):
    gold = result.question.get('metadata.answer')

    scores = []
    for answer in result:
        score = rouge_scorer.score(gold, answer.text)
        scores.append(score['rouge1'].recall)

    inds = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    # print(f"Best answer for {res.question.text}: {res[inds[0]]['id']}, recall: {scores[inds[0]]}")
    # return {result.question.id: {"best": result[inds[0]]['id'], "score": scores[inds[0]]}}
    res = []
    for ansid in inds:
        if scores[ansid] >= thr:
            res.append({'id': result[ansid]["id"], 'score': scores[ansid]})
    return res

In [31]:
compute_rouge_matches(res, 0.9)

[{'id': '451813522832071401-0-1261', 'score': 1.0}]

In [16]:
compute_rouge_matches(results[3])

[{'id': '451813522833601023-0-823', 'score': 1.0}]

In [69]:
import json

questions = [json.loads(s) for s in open("benchmark/zdocs/questions_all_fixed.jsonl").readlines()]

FileNotFoundError: [Errno 2] No such file or directory: 'benchmark/zdocs/questions_all_fixed.jsonl'

In [95]:
questions[0]['metadata']['gold-urls']

'https://www.ibm.com/docs/en/zos/2.5.0?topic=considerations-data-set-encryption'

In [118]:
from tqdm.notebook import tqdm
from docuverse.utils import get_orig_docid
outq = questions.copy()
not_found = []

def append(ll, vals):
    for v in vals:
        if v not in ll:
            ll.append(v)
for i, q in tqdm(enumerate(outq), total=len(outq)):
    gold_urls = q['metadata']['gold-urls'].split(";")
    relevant = []
    for gold_url in gold_urls:
        gold_url = normalize_url(q['metadata']['gold-urls'])
        rels = [get_orig_docid(p) for p in url2id[gold_url]] if gold_url in url2id else []
        append(relevant, rels)
    matches = compute_rouge_matches(results[i])
    if len(matches) > 0:
        # q['metadata']['relevant'] = [get_orig_docid(m['id']) for m in matches]
        append(relevant, [get_orig_docid(m['id']) for m in matches])
        # relevant += [get_orig_docid(m['id']) for m in matches if get_orig_docid(m['id']) not in relevant]
        q['metadata']['rouge_scores'] = [m['score'] for m in matches]
    else:
        q['metadata']['rouge_scores'] = []
        # gurl = q['metadata']['gold-urls']
        # relevant = [url2id[gurl].keys()] if gurl in url2id else []
        # if not relevant:
        #     gurl = normalize_url(gurl)
        #     relevant = [url2id[gurl].keys()] if gurl in url2id else []
    if not relevant:
        not_found.append(i)
    q['metadata']['relevant'] = relevant

  0%|          | 0/510 [00:00<?, ?it/s]

In [120]:
not_found[0]

13

In [121]:
questions[not_found[0]].as_dict()

{'metadata.answer': 'You can use the DEVSERV MVS command with Query Host Access (QHA) parameter to request host access information to be returned for a volume, depending on the type of device: \r\n1.ﾂ For DASD, issue DEVSERV QDASD,device,QHA\r\n2. For TAPE, issue DEVSERV QTAPE,device,QHA',
 '_id': 'dfsms_32',
 'relevant': [''],
 'metadata': {'gold-urls': 'https://www.ibm.com/docs/en/zos/2.5.0?topic=command-using-devserv-qdasd-option;https://www.ibm.com/docs/en/zos/2.5.0?topic=command-using-devserv-qtape-option',
  'answer': 'You can use the DEVSERV MVS command with Query Host Access (QHA) parameter to request host access information to be returned for a volume, depending on the type of device: \r\n1.ﾂ For DASD, issue DEVSERV QDASD,device,QHA\r\n2. For TAPE, issue DEVSERV QTAPE,device,QHA',
  'fname': '/Users/vineet/data/systemz/data/predicted_ans_with_analysis_20250121/DFSMS_QA-Ceci_v0_1.res.csv',
  'rouge_scores': [],
  'relevant': []},
 'answers': [''],
 'passages': ''}

In [26]:
open("benchmark/zdocs/missing_urls.new.txt", "w").writelines([q['metadata']['gold-urls']+"\n" for q in outq if q['metadata']['relevant']==[] and q['metadata']['answer']!=""])

In [23]:
# Fix the questions' urls: remove tags
import json
qfile = "benchmark/zdocs/questions_all_fixed.jsonl"
qfile_fixed = "benchmark/zdocs/questions_all_fixed_new.jsonl"
qs = open(qfile).readlines()
with open(qfile_fixed, "w") as out:
    for line in qs:
        q = json.loads(line)
        url = q['metadata']['gold-urls']
        pos = url.find('#')
        if pos >= 0:
            q['metadata']['gold_urls'] = url[:pos]
        out.write(json.dumps(q)+"\n")

Fix the questions to have the 'latest' tag (according to Jaydeep, this was agreed with the Z team).

In [9]:
questions=engine.read_questions()

Reading benchmark/zdocs/questions_all.jsonl:: 510it [00:00, 295536.76it/s]


In [52]:
qid=0
url=questions[qid]['metadata']['gold-urls']
norm_url = normalize_url(url)
print(f"Url in url2id: {norm_url in url2id}")
print(f"Normalized url in url2id: {norm_url in url2id}")
found=compute_rouge_matches(results[qid], 0.9)
found_id=found[0]['id']
print(found_id)
print(f"Id for results[{qid}]: {found_id}, \nurl:{data[id2pos[found_id]]['metadata']['url']}")

Url in url2id: False
Normalized url in url2id: False
451813522832071401-0-1261
Id for results[0]: 451813522832071401-0-1261, 
url:https://www.ibm.com/docs/en/zos/3.1.0?topic=considerations-data-set-encryption
